In [93]:
# Import packages
 # import fuzzywizzy?
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import pymongo
import string
import nltk
import re
import os
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')



src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

# helper functions
from d02_processing.cleaning_signatures import sorted_signatures
from d02_processing.cleaning_signatures import cleaned_signatures
from d01_utils.mongo_cursor_creator import mongo_cursor

# Load the "autoreload" extension
%load_ext autoreload

# reload modules so that as you change code in src, it gets loaded
%autoreload

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# import the unique signatures from the database
sigs_from_cursor = mongo_cursor()
sigs_from_cursor[0:5]

['',
 '2c/3a, fine texture, normal porosity, medium-to-thick\nCo wash: Vo5 Extra Body\nConditioners: Deva One Condition, GVP CB, Loreal Evercurl\nStylers: Devacurl ArcAngel, KCCC, AG Recoil, Iso Bouncy Creme, Ecostyler Krystal, CCCCL, KCKT\nLow Poo: Devacurl NoPoo, GTTT\nHair likes: plopping, diffusing, SMaster\'s, coconut and argan oil, honey, protein\n"What makes a woman unforgettable? Her mind...surrounded by lots of naturally curly hair"',
 'True 3B\ncoarse, overly porous, normal elasticity, thick\nOn the long transitioning road...\nDiscovered Curly: November 24, 2009:toothy3:\nHates: humectants, protein, rain, wind, humidity, plopping\nLoves: moisture, ecostyler, sallys conditioning balm\n\n:wav:\n\n\n\n\n\n\n\n\n',
 "Silence grows\nMy feelings flow\nI'm dreaming now\nOf all the things I know\n",
 "I've learned that no matter what happens, or how bad it seems today, life does go on, and it will be better tomorrow.\n\n3 b/c shoulder length hair. Black-brown color. It's always mista

In [5]:
# sort the signatures
raw_characteristics_df = sorted_signatures(sigs_from_cursor)
raw_characteristics_df.head()

,curl_pattern,density,porosity,texture,products
0,NaN,NaN,NaN,NaN,
1,2c,thick,normal,fine,to co wash vo5 extra body conditioners deva ...
2,3a,thick,normal,fine,to co wash vo5 extra body conditioners deva ...
3,3b,thick,NaN,NaN,true coarse overly porous normal elasticity on...
4,NaN,thin,NaN,NaN,silence grows my feelings flow i m dreaming no...


In [6]:
cleaned_df = cleaned_signatures(raw_characteristics_df)
cleaned_df.head()

,curl_pattern,density,porosity,texture,products
1,2c,thick,normal,fine,to co wash vo5 extra body conditioners deva ...
2,3a,thick,normal,fine,to co wash vo5 extra body conditioners deva ...
3,3b,thick,NaN,NaN,true coarse overly porous normal elasticity on...
4,NaN,thin,NaN,NaN,silence grows my feelings flow i m dreaming no...
9,4a,NaN,NaN,NaN,toothy4 mix toothy4 bc 09 28 12 occasion9 gro...


In [54]:
# make a data fram with no nulls in the different characteristics
curl_pattern_no_nan_df = cleaned_df[pd.notnull(cleaned_df['curl_pattern'])]
curl_pattern_no_nan_df['curl_catagory'] = pd.Categorical(curl_pattern_no_nan_df.curl_pattern)
curl_pattern_no_nan_df["curl_catagory"] = curl_pattern_no_nan_df["curl_catagory"].cat.codes
print(curl_pattern_no_nan_df.describe())
curl_pattern_no_nan_df.head()

       curl_catagory
count    7405.000000
mean        4.342201
std         1.686252
min         0.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         9.000000


,curl_pattern,density,porosity,texture,products,curl_catagory
1,2c,thick,normal,fine,to co wash vo5 extra body conditioners deva ...,3
2,3a,thick,normal,fine,to co wash vo5 extra body conditioners deva ...,4
3,3b,thick,NaN,NaN,true coarse overly porous normal elasticity on...,5
9,4a,NaN,NaN,NaN,toothy4 mix toothy4 bc 09 28 12 occasion9 gro...,7
10,3c,NaN,NaN,NaN,toothy4 mix toothy4 bc 09 28 12 occasion9 gro...,6


In [19]:
cleaned_df['curl_catagory'] = pd.Categorical(cleaned_df.curl_pattern)
cleaned_df.dtypes


curl_pattern       object
density            object
porosity           object
texture            object
products           object
curl_catagory    category
dtype: object

In [21]:
cleaned_df["curl_catagory"] = cleaned_df["curl_catagory"].cat.codes
cleaned_df.head()

,curl_pattern,density,porosity,texture,products,curl_catagory
1,2c,thick,normal,fine,to co wash vo5 extra body conditioners deva ...,3
2,3a,thick,normal,fine,to co wash vo5 extra body conditioners deva ...,4
3,3b,thick,NaN,NaN,true coarse overly porous normal elasticity on...,5
4,NaN,thin,NaN,NaN,silence grows my feelings flow i m dreaming no...,-1
9,4a,NaN,NaN,NaN,toothy4 mix toothy4 bc 09 28 12 occasion9 gro...,7


In [24]:
cleaned_df.products

1          to co wash vo5 extra body conditioners deva ...
2          to co wash vo5 extra body conditioners deva ...
3        true coarse overly porous normal elasticity on...
4        silence grows my feelings flow i m dreaming no...
9         toothy4 mix toothy4 bc 09 28 12 occasion9 gro...
10        toothy4 mix toothy4 bc 09 28 12 occasion9 gro...
11        afro i love4 water he my baby daddy is the de...
12        afro i love4 water he my baby daddy is the de...
13       hair stats i guess  shoulder length tends to f...
14       hair stats i guess  shoulder length tends to f...
15       bc d 6 5 2010 last relaxed 5 16 2009 mostly pr...
16                                             on the side
17        sigpic sigpic littlemisscurls shutterfly com ...
18        sigpic sigpic with little s thrown in to make...
19        sigpic sigpic with little s thrown in to make...
20       i k i m a definitely a corkscrew currently on ...
21        wavy baby hair low low poo lush karma komba h.

In [70]:
# Check on frequency of n-grams

def generate_ngrams(s, n):
    # Convert to lowercases
    s = s.lower()
    
    # Replace all none alphanumeric characters with spaces
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    
    # Break sentence in the token, remove empty tokens
    tokens = [token for token in s.split(" ") if token != ""]
    
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

In [73]:
# all quimtagrams in the product set

result = [generate_ngrams(x, 5) for x in curl_pattern_no_nan_df['products']]
result

[['to co wash vo5 extra',
  'co wash vo5 extra body',
  'wash vo5 extra body conditioners',
  'vo5 extra body conditioners deva',
  'extra body conditioners deva one',
  'body conditioners deva one condition',
  'conditioners deva one condition gvp',
  'deva one condition gvp cb',
  'one condition gvp cb loreal',
  'condition gvp cb loreal evercurl',
  'gvp cb loreal evercurl stylers',
  'cb loreal evercurl stylers devacurl',
  'loreal evercurl stylers devacurl arcangel',
  'evercurl stylers devacurl arcangel kccc',
  'stylers devacurl arcangel kccc ag',
  'devacurl arcangel kccc ag recoil',
  'arcangel kccc ag recoil iso',
  'kccc ag recoil iso bouncy',
  'ag recoil iso bouncy creme',
  'recoil iso bouncy creme ecostyler',
  'iso bouncy creme ecostyler krystal',
  'bouncy creme ecostyler krystal ccccl',
  'creme ecostyler krystal ccccl kckt',
  'ecostyler krystal ccccl kckt low',
  'krystal ccccl kckt low poo',
  'ccccl kckt low poo devacurl',
  'kckt low poo devacurl nopoo',
  'low p

In [85]:
# put all the ngrams in one list rather than a list of lists
quintagram = []
for r in result:
    for x in r:
        quintagram.append(x)
quintagram[:50]  

['to co wash vo5 extra',
 'co wash vo5 extra body',
 'wash vo5 extra body conditioners',
 'vo5 extra body conditioners deva',
 'extra body conditioners deva one',
 'body conditioners deva one condition',
 'conditioners deva one condition gvp',
 'deva one condition gvp cb',
 'one condition gvp cb loreal',
 'condition gvp cb loreal evercurl',
 'gvp cb loreal evercurl stylers',
 'cb loreal evercurl stylers devacurl',
 'loreal evercurl stylers devacurl arcangel',
 'evercurl stylers devacurl arcangel kccc',
 'stylers devacurl arcangel kccc ag',
 'devacurl arcangel kccc ag recoil',
 'arcangel kccc ag recoil iso',
 'kccc ag recoil iso bouncy',
 'ag recoil iso bouncy creme',
 'recoil iso bouncy creme ecostyler',
 'iso bouncy creme ecostyler krystal',
 'bouncy creme ecostyler krystal ccccl',
 'creme ecostyler krystal ccccl kckt',
 'ecostyler krystal ccccl kckt low',
 'krystal ccccl kckt low poo',
 'ccccl kckt low poo devacurl',
 'kckt low poo devacurl nopoo',
 'low poo devacurl nopoo gttt',
 'p

In [91]:
quintagram_df = pd.DataFrame({'quintagrams': quintagram})
quintagram_df.quintagrams.value_counts()

co wash suave naturals coconut                   86
shea moisture curl enhancing smoothie            47
as i am coconut cowash                           36
leave in kinky curly knot                        34
trader joe s tea tree                            33
in kinky curly knot today                        32
suave naturals coconut rinse out                 30
co wash as i am                                  29
giovanni tea tree triple treat                   28
herbal essences set me up                        28
order of the curly crusaders                     26
co wash suave tropical coconut                   25
gvp conditioning balm leave in                   25
leave in giovanni direct leave                   23
as i am leave in                                 23
the order of the curly                           22
low poo giovanni 50 50                           21
still trying to figure out                       21
in giovanni direct leave in                      21
joe s tea tr

In [90]:
# Here is a different less ordered way to do it
Counter(quintagram)

Counter({'to co wash vo5 extra': 2,
         'co wash vo5 extra body': 2,
         'wash vo5 extra body conditioners': 2,
         'vo5 extra body conditioners deva': 2,
         'extra body conditioners deva one': 2,
         'body conditioners deva one condition': 2,
         'conditioners deva one condition gvp': 2,
         'deva one condition gvp cb': 2,
         'one condition gvp cb loreal': 2,
         'condition gvp cb loreal evercurl': 2,
         'gvp cb loreal evercurl stylers': 2,
         'cb loreal evercurl stylers devacurl': 2,
         'loreal evercurl stylers devacurl arcangel': 2,
         'evercurl stylers devacurl arcangel kccc': 2,
         'stylers devacurl arcangel kccc ag': 2,
         'devacurl arcangel kccc ag recoil': 2,
         'arcangel kccc ag recoil iso': 2,
         'kccc ag recoil iso bouncy': 2,
         'ag recoil iso bouncy creme': 2,
         'recoil iso bouncy creme ecostyler': 2,
         'iso bouncy creme ecostyler krystal': 2,
         'bouncy

In [68]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 5), stop_words='english')
features = tfidf.fit_transform(curl_pattern_no_nan_df.products).toarray()
labels = curl_pattern_no_nan_df.curl_catagory
features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [56]:
# df['category_id'] = df['Product'].factorize()[0]
# from io import StringIO
# category_id_df = df[['Product', 'category_id']].drop_duplicates().sort_values('category_id')
# category_to_id = dict(category_id_df.values)
# id_to_category = dict(category_id_df[['category_id', 'Product']].values)

curl_catagory_df = curl_pattern_no_nan_df[['curl_pattern', 'curl_catagory']].drop_duplicates().sort_values('curl_catagory')
curl_cat_key_dict = dict(curl_catagory_df.values)
curl_pattern_key_dict = dict(curl_catagory_df[['curl_catagory', 'curl_pattern']].values)
print(curl_cat_key_dict)
print(curl_pattern_key_dict)

{'1c': 0, '2a': 1, '2b': 2, '2c': 3, '3a': 4, '3b': 5, '3c': 6, '4a': 7, '4b': 8, '4c': 9}
{0: '1c', 1: '2a', 2: '2b', 3: '2c', 4: '3a', 5: '3b', 6: '3c', 7: '4a', 8: '4b', 9: '4c'}


In [69]:
N = 2
for curl_pattern, curl_cat in sorted(curl_cat_key_dict.items()):
    features_chi2 = chi2(features, labels == curl_cat)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    trigrams = [v for v in feature_names if len(v.split(' ')) == 3]
    quadgrams = [v for v in feature_names if len(v.split(' ')) == 4]
    quintagrams = [v for v in feature_names if len(v.split(' ')) == 5]
    print("# '{}':".format(curl_pattern))
    #print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
    #print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))
    print("  . Most correlated trigrams:\n. {}".format('\n. '.join(trigrams[-N:])))
    print("  . Most correlated quadgrams:\n. {}".format('\n. '.join(quadgrams[-N:])))
    print("  . Most correlated quadgrams:\n. {}".format('\n. '.join(quintagrams[-N:])))

# '1c':
  . Most correlated trigrams:
. jessicurl cleansing cream
. psalm 139 14
  . Most correlated quadgrams:
. poo mop hydrating shampoo
. tresemme naturals moisturizing conditioner
  . Most correlated quadgrams:
. tresemme naturals nourishing moisture conditioner
. suave naturals tropical coconut ro
# '2a':
  . Most correlated trigrams:
. healthy sexy hair
. ro ss caitlin
  . Most correlated quadgrams:
. normal elasticity low poo
. la looks mega hold
  . Most correlated quadgrams:
. stylers herbal essences totally twisted
. low poo cj daily fix
# '2b':
  . Most correlated trigrams:
. beauticurls li curl
. shampoo body shop
  . Most correlated quadgrams:
. wash vo5 kiwi lime
. yes cucumbers color protection
  . Most correlated quadgrams:
. low poo giovanni 50 50
. low poo devacurl low poo
# '2c':
  . Most correlated trigrams:
. coarse normal elasticity
. curly just wavy
  . Most correlated quadgrams:
. moisture coconut hibiscus hold
. li kinky curly knot
  . Most correlated quadgram

In [58]:
X = cleaned_df[['density', 'porosity', 'products', 'texture']]
y = cleaned_df.curl_catagory
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

TypeError: take_nd() got an unexpected keyword argument 'axis'

In [64]:
#Test some models now

X_train, X_test, y_train, y_test = train_test_split(curl_pattern_no_nan_df['products'], curl_pattern_no_nan_df['curl_pattern'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [94]:

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0)]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [95]:
cv_df.groupby('model_name').accuracy.mean()

model_name
LinearSVC                 0.280882
LogisticRegression        0.319106
MultinomialNB             0.310471
RandomForestClassifier    0.272654
Name: accuracy, dtype: float64